# Lab and Homework 1: Web Scraping

**In class**: *Tuesday, February 5, 2019*  
**Homework Due** : *5PM, Tuesday, February 12, 2019*

# Learning Goals

* Learn about the utility, tools and techniques of data scraping.
* Learn about HTML and its syntactic structure.
* Learn how HTML files are parsed into a form that is amenable to easy analysis and extraction.
* Brush up on pandas Dataframes, particularly indexing and aggregation.

Like most of units in this course, this notebook consists of two parts. 

The first part introduces basic concepts. We'll explore these concepts during class. The second part is for homework, which builds upon and extends the things you learned in class.

# Web Scraping

Data is the *raison d'être* of data science. No data no science. (But also...[No Data No Cry](https://www.youtube.com/watch?v=pHlSE9j5FGY)!) 

Thus acquiring data is the first step to doing anything in this discipline. 

Unfortunately, the required data often isn't readily available in an easy-to-read zipfile or database, ready to be exploited. You generally have to find it, get it and shape it to your needs.

But fortunately, the web is a rich source of information and we 
In this unit we explore data scraping with a specific application in mind. You've been asked to assemble some 

# Problem Statement

Let's say you're doing research on corporate governance in the Fortune 100 and want to determine compensation rates of the board of directors and their potential conflicts. 

How might you go about getting the data for the companies in the list? Perhaps if you have a subscription to Bloomberg, then you could download a CVS. But if you don't, your best bet is scrape public pages on the web.

In this lab and homework we'll learn how to scrape Reuters finance pages to create our own data set ready for analysis. We'll do the scraping in class and the analysis for homework.


# Scraping Summarized

In principle, web scraping is simple and involves the following steps:

1. **Inspect** the web page. This will give you a sense the overall structure of the page and where the relevant information resides. To do this, you will open the web page in a browser and then view the page *source*. We explain how below.

1. **Retrieve** a web page over HTTP as text. This text will be formatted as HTML, the language of the World Wide Web. Your browser interprets the content of an HTML file and renders it to screen. (This is a gross over-simplification. Many sites today are *dynamic* apps written in Javascript. They retrieve data over an API and render it programmatically on screen. Still, a lot of information resides in *static* HTML and the techniques described here remain quite useful.)

1. **Parse** the retrieved HTML text into a form that can easily scanned and operated upon. Fortunately, we don't have to do this ourselves. Great and powerful Python libraries such as [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) and [Scrapy](https://scrapy.org) already exist. In this unit, we'll use Beautiful Soup. But I encourage you to take a look at Scrapy. It's in many ways more powerful and flexible.

1. **Search** the parsed HTML for the information we're interested in.

1. **Pull** the relevant data out of the HTML, reformat it into a form that meets our needs, and save it for later analysis. In this lab, we'll insert our data into a [Pandas](https://pandas.pydata.org) Dataframe. In future units, we'll likely save to a database.



# Part I: In Class Lab

For the in-class part of this unit, we are going to do the following:

1. Retrieve the stock symbols for the companies in the S&P 100. To do this, we'll scrape a Wikipedia page.
2. With the S&P symbols in hand, we'll retrieve the board of directors by scraping a page for each company on the Reuters Financial site. You will use the skills developed in scraping the Wikipedia page to get the boards.
3.You will next use your Pandas chops to compute the average age of each company board. Easy Peasy!

## Imports

These are the packages we'll use in this lab. Please use the [Installation]() notebook to set up all the modules we'll be using this semester. We'll add new packages in the Installation notebook as they are needed.

## Notes

1. Lexical analysis
2. Parsing

### Lab

1. Wikipedia
2. Reuters Tinancid

In [1]:
#Packages to install

# pretty printer
import pprint

# set up the pretty printer
pp = pprint.PrettyPrinter(indent=4)

# BeautifulSoup for scraping
from bs4 import BeautifulSoup

# for making HTTP requests
import requests

# Pandas/numpy for data manipulation
import pandas as pd
import numpy as np

## Retrieve the S&P 100 Stock Symbols

In order to investigate the board of directors at each company in the S&P 100, we're going to need their stock symbols. Here we'll scrape the [Wikipedia page](https://en.wikipedia.org/wiki/S%26P_100) for the S&P 100.

You should load the [Wiki](https://en.wikipedia.org/wiki/S%26P_100) page in a browser and study it's source.

The listing of S&P companies is in a `<table>` element. We'll use BeautifulSoup to parse the HTML into a parse tree, a hierarchical representation of the page, which makes it much easier to scan for the elements we want. BeautifulSoup will scan the parse tree, find the table in question, and scan each row for the company and symbol. We'll put the extracted data in a Pandas DataFrame for later analysis and manipulation.

In [2]:
# The URL for the Wikipedia page we're scraping
WIKI_URL = 'https://en.wikipedia.org/wiki/S%26P_100'

# Retrieve the page
wiki_page = requests.get(WIKI_URL).text



We now have the wiki page in the variable `wiki_page`. We should print it directly the output will be unstructured and therefore quite hard to read.

Let's use Beautiful Soup to read the text into a parse tree and then render the parse tree to screen like this:

In [3]:
# parse the HTML text into a tree
soup = BeautifulSoup(wiki_page, 'html.parser')

# print the tree to screen
#print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   S&amp;P 100 - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"S\u0026P_100","wgTitle":"S\u0026P 100","wgCurRevisionId":878125111,"wgRevisionId":878125111,"wgArticleId":2658424,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["American stock market indices","S\u0026P Dow Jones Indices"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","J

Whoa!!! There's a lot going on here. 

But if as you poke around, you find that the ticker symbols are in an HTML `<table>`:

```erb
<table class="wikitable sortable">
       <tbody>
        <tr>
         <th>
          Symbol
         </th>
         <th>
          Name
         </th>
        </tr>
        <tr>
         <td>
          AAPL
         </td>
         <td>
          <a href="/wiki/Apple_Inc." title="Apple Inc.">
           Apple Inc.
          </a>
         </td>
        </tr>
```

The body of the table has two columns. The symbol is in the left column and the company name on the right. 

Let's use BeautifulSoup to extract the table from the parse tree. 

We're looking for the table with CSS classes `wikitable` and `sortable`.

In [4]:
# extract the table containin the S&P companies
sandp_table = soup.find('table', {"class" : "wikitable sortable"})


With our table of S&P companies in hand, we can traverse it row by row to retrieve each company and its symbol. Here's how:

In [78]:
# snps array will hold an array of tuples of the form (Symbol, Name)
snps = []

# scan the table for each row ('tr' is the HTML tag for a table row)
for row in sandp_table.find_all('tr'):
    
    # scan the row for table cells ('td' is the tag for table data)
    cols = row.find_all('td')
    
    if len(cols) == 2: # skip the header row
        snps.append((cols[0].text.strip(), cols[1].text.strip()))

# convert the array of tuples into a Pandas DataFrame        
snps_df = pd.DataFrame(snps, columns=['Symbol', 'Name'])

snps_df.head()

,Symbol,Name
0,AAPL,Apple Inc.
1,ABBV,AbbVie Inc.
2,ABT,Abbott Laboratories
3,ACN,Accenture plc
4,AGN,Allergan plc


## Lab Problem 1

We now have the S&P 100 stock symbols. But we're far from done. What we want is information about the boards of each company, which we'll do in this Lab.

To do this, iterate over the snps_df DataFrame created above and scrape the Reuters Financials page for company boards.

As one example, here's the Reuters page for [Apple](http://www.reuters.com/finance/stocks/company-officers/AAPL) (AAPL). Open the page in a browser and study the HTML source to find where the BoD resides. Then flesh out the skeleton code below.

Organize each row of the table into the following columns:

1. **Symbol**: the ticker symbol of the company.
1. **Name**: the name of the board member.
1. **Link**: a link to the detail page for the board member. Hint: the link is embedded in an `<a>` element of the cell that contains the board member's name.
1. **Age**: the board member's age encoded as an `integer`. (We want to run aggregate functions on the age).
1. **Year**: the year the member started.

Create a DataFrame called `df` with the above column names.

In [6]:

REUTERS_BASE_URL = 'http://www.reuters.com/finance/stocks/company-officers/'

symbol_array = snps_df.Symbol.values

# board_members will hold an array of tuples, one for each board member
board_members = []

# for simplicity only look at the first five companies in class
for sym in symbol_array:
    """This section gets the HTML table"""
    link = REUTERS_BASE_URL +'{}'.format(sym)
    txt = requests.get(link).text
    soup = BeautifulSoup(txt, 'html.parser')
    table = soup.find('table')
    
    val_names = [col.get_text().strip().split('\xa0') for col in table.find_all('td')]
    link = [w.get('href') for w in table.find_all('a')]
    
    names = []
    age = []
    year = []
    
    url = 'http://www.reuters.com'
    for i in range(0, len(val_names), 4):
        names.append(val_names[i][0] +' '+val_names[i][1])
        age.append(val_names[i+1][0])
        year.append(val_names[i+2][0])
        
    for j in range(0, len(age)):
        board_members.append((sym, names[j], url +link[j], age[j], year[j]))
    
# for (index, co) in snps_df.iterrows():
#    sym = co['Symbol']

cols_names = ['Symbol', 'Name', 'Link', 'Age', 'Year']    
df = pd.DataFrame(board_members, columns=cols_names)

# return and print df
df.head()

,Symbol,Name,Link,Age,Year
0,AAPL,Art Levinson,http://www.reuters.com/finance/stocks/officer-...,67,2011
1,AAPL,Timothy Cook,http://www.reuters.com/finance/stocks/officer-...,57,2011
2,AAPL,Luca Maestri,http://www.reuters.com/finance/stocks/officer-...,54,2014
3,AAPL,Katherine Adams,http://www.reuters.com/finance/stocks/officer-...,53,2017
4,AAPL,Phil Schiller,http://www.reuters.com/finance/stocks/officer-...,57,2002


## Lab Problem 2

Notice that the `Symbol` column in the `df` DataFrame above has multiple rows for each company--one row for each board member.

We want to retieve and aggregate over individual companies. One way to do this is to create a Pandas *multiindex* on `Symbol` and `Name`. Create an `inplace` index, i.e., one that modifies the `df` DataFrame instead of returning a new one.

In [7]:
# Lab Problem 2
# Create a multiindex as described above

df.set_index(['Symbol', 'Name'], inplace=True)
df.tail()

Link  \
Symbol Name                                                                    
XOM    Steven Kandarian    http://www.reuters.com/finance/stocks/officer-...   
       Douglas Oberhelman  http://www.reuters.com/finance/stocks/officer-...   
       Samuel Palmisano    http://www.reuters.com/finance/stocks/officer-...   
       Steven Reinemund    http://www.reuters.com/finance/stocks/officer-...   
       William Weldon      http://www.reuters.com/finance/stocks/officer-...   

                          Age  Year  
Symbol Name                          
XOM    Steven Kandarian    67  2018  
       Douglas Oberhelman  65  2015  
       Samuel Palmisano    66  2016  
       Steven Reinemund    70  2016  
       William Weldon      69  2013

## Lab Problem 3

Use the indexed `df` to retrieve the entries for `GOOG`.

Hint: use `df.loc`

In [8]:
# Lab problem 3

# your code here
df.loc['GOOG']

,Link,Age,Year
Name,,,
John Hennessy,http://www.reuters.com/finance/stocks/officer-...,65,2018
Sergey Brin,http://www.reuters.com/finance/stocks/officer-...,44,2015
Lawrence Page,http://www.reuters.com/finance/stocks/officer-...,45,2015
Ruth Porat,http://www.reuters.com/finance/stocks/officer-...,60,2015
Sundar Pichai,http://www.reuters.com/finance/stocks/officer-...,46,2017
David Drummond,http://www.reuters.com/finance/stocks/officer-...,55,2015
Diane Greene,http://www.reuters.com/finance/stocks/officer-...,62,2015
Eric Schmidt,http://www.reuters.com/finance/stocks/officer-...,62,2018
L. John Doerr,http://www.reuters.com/finance/stocks/officer-...,66,2016


## Lab Problem 4

Compute the mean age of each board in the S&P 100. Which company has the oldest board? Which the youngest?

In [9]:
# Lab problem 4
# compute and print the mean age of each board
# your code here
symbol_array
df.Age = pd.to_numeric(df.Age)
lst = {}
for sym in symbol_array:
    lst[sym] = round(df.loc[sym]['Age'].mean())
    
for key, value in lst.items():
    print('The average age for company {} is {}'.format(key, value))

The average age for company AAPL is 59
The average age for company ABBV is 58
The average age for company ABT is 60
The average age for company ACN is 56
The average age for company AGN is 57
The average age for company AIG is 61
The average age for company ALL is 59
The average age for company AMGN is 61
The average age for company AMZN is 59
The average age for company AXP is 59
The average age for company BA is 58
The average age for company BAC is 62
The average age for company BIIB is 58
The average age for company BK is 55
The average age for company BKNG is 58
The average age for company BLK is 57
The average age for company BMY is 59
The average age for company BRK.B is 72
The average age for company C is 60
The average age for company CAT is 59
The average age for company CELG is 63
The average age for company CHTR is 55
The average age for company CL is 61
The average age for company CMCSA is 61
The average age for company COF is 56
The average age for company COP is 59
The a

In [10]:
# Find the boards with the maximum and minimum average ages
print('{} is the company with youngest board members, averaging {} years old'.format(min(lst, key=lst.get) , 
                                                                           str(lst[min(lst, key=lst.get)])))
print('{} is the company with oldest board members, averaging {} years old'.format(max(lst, key=lst.get), 
                                                                         str(lst[max(lst, key=lst.get)])))

FB is the company with youngest board members, averaging 50 years old
BRK.B is the company with oldest board members, averaging 72 years old


# Homework

*40 points total*

In the homework you will build upon the lab work to retrieve data about each board member and then perform some basic aggregations.

## Homework Problem 1

*10 points*

Retrieve the total compensation of each board member and put the results in a DataFrame of the following columns:

1. **Symbol**: The company stock symbol.
1. **Name**: Board Member's name.
1. **Total**: Total yearly compensation for the member.

To do this problem you'll use the `link`attribute in the `df` DataFrame from the lab.

You should decide how to index the DataFrame to best utilize it for subsequent problems.

In [19]:
## Homework problem 1
compensation_table = None;

# Your code here

tab = [] # empty list for grouping Symbol, Name and Total as tuple

# populating the list tab
for sym, name in df.index: 
    
    # Soup cooking from HTML
    link = df.loc[sym, name].Link
    txt = requests.get(link).text
    soup = BeautifulSoup(txt, 'html.parser')
    
    tables = soup.find_all('table') 
    
     # List appending of tupples
    if len(tables) == 3:
        tds = tables[0].find_all('td', {"class":"data center"})[4].text.replace(',', '')
        if tds == '--':
            tab.append((sym, name, np.nan))
        else:
            tab.append((sym, name, tds))
            
    elif len(tables)==1:
        tab.append((sym, name, np.nan))
        
        
# Data Frame setup
col_nms = ['Symbol', 'Name', 'Total']
compensation_table = pd.DataFrame(tab, columns = col_nms)

# Change of indexes
compensation_table.set_index(['Symbol', 'Name'], inplace=True)

# Total, column pulishing
compensation_table['Total'] = pd.to_numeric(compensation_table['Total'])

compensation_table.head()

Total
Symbol Name                       
AAPL   Art Levinson       556362.0
       Timothy Cook     12825100.0
       Luca Maestri     24141600.0
       Katherine Adams         NaN
       Phil Schiller    11677200.0

In [20]:
compensation_table.tail()

Total
Symbol Name                        
XOM    Steven Kandarian         NaN
       Douglas Oberhelman  336918.0
       Samuel Palmisano    346918.0
       Steven Reinemund    346918.0
       William Weldon      336918.0

In [27]:
compensation_table.shape

(2242, 1)

In [48]:
de = data.loc[data.index.get_level_values('Name')=='William Weldon']
de

,,Link,Age,Year,Total
Symbol,Name,,,,
CVS,William Weldon,http://www.reuters.com/finance/stocks/officer-...,69.0,2013,280000.0
JPM,William Weldon,http://www.reuters.com/finance/stocks/officer-...,69.0,2005,442500.0
XOM,William Weldon,http://www.reuters.com/finance/stocks/officer-...,69.0,2013,336918.0


## Homework Problem 2

*10 points*

Compute the mean compensation for each company and put the results in a DataFrame with the following columns:

1. **Symbol**: The company stock symbol
1. **Compensation**: Mean board compensation
1. **Age**: Mean board age

Notice that you're asked to include the mean age. This suggests that you will `join` two tables together. How will you index these two tables in order to compute the result elegantly and simply?

In [80]:
# Homework problem 2
company_compansation_table = None

# Your code here

data = df.join(compensation_table, how='outer')

lsts = []

for sym in symbol_array:
    lsts.append((sym, data.loc[sym]['Total'].mean(skipna=True), round(data.loc[sym]['Age'].mean(skipna=True))))

cl_nms = ['Symbol', 'Compensation', 'Age']

company_compensation_table = pd.DataFrame(lsts, columns=cl_nms)

print(company_compensation_table.shape)

company_compensation_table.set_index(['Symbol'], inplace=True)

company_compensation_table.head()

(102, 3)


,Compensation,Age
Symbol,,
AAPL,1.125967e+07,59
ABBV,5.104063e+06,58
ABT,4.110890e+06,60
ACN,3.168184e+06,56
AGN,5.820609e+06,57


## Homework Problem 3

*10 points*

You might have noticed that some people sit on more than one board. This is sometimes an indication of potential conflicts of interest. In this problem, you will identify members that on serve multiple boards.

The output will be a DataFrame with columns:

1. **Name**: Board member.
1. **Symbol**: Symbol of company
1. **Count**: Number of companies the board member serves

This problem requires you to revisit your Pandas aggregation chops. You're essentially taking `df` from before and applying a transformation. One or two lines of code if you leverage pandas; many lines of procedural code if you don't.

In [77]:
# Homework Problem 3

conflicts_table = None

# your code here
lstss = []
for i in range(data.shape[0]):
    lstss.append((data.index[i][1], data.index[i][0]))

to = ['Name', 'Symbol']
tot = pd.DataFrame(lstss, columns=to)
tata = pd.DataFrame(tot['Name'].value_counts())
tata.columns = ['Count']
tot.set_index(['Name'], inplace=True)
conflicts_table = tot.join(tata, how='outer')
conflicts_table.index.name = 'Name'

print(conflicts_table.shape)
conflicts_table.head()

(2242, 2)


,Symbol,Count
Name,,
A. Brooke Seawell,NVDA,1
A. Eugene Washington,JNJ,1
A. Louise Pentland,PYPL,1
A. Robert Bailey,AGN,1
Aaron Abramovitz,SO,1


## Homework Problem 4

* 10 points *

Take the quick poll posted on Slack. [Help me help you!](https://www.youtube.com/watch?v=l1B1_jQnlFk) 

Seriously, by giving me feedback, I'll be able to improve the lectures, labs, and homeworks. We'll be better for it.

Free 10 points, right?

# Summary

We've learned the basics of web scraping in this unit. Hopefully, you've added a new tool to your belt. 

But really, we've barely *scraped* the surface of web scraping!

The approach is fraught with complications. For one, it's brittle. What happens if some developer changes the structure of the pages you've scraped? Everything breaks next time. 

For another, we addressed the simplest use case: **static** HTML pages. Many modern sites are **dynamic** apps, built with Javascript that runs in the browser. Dynamic apps are harder to scrape but there are tools for the job. One such tool is [Selenium](https://www.seleniumhq.org/). It basically programmatically simulates interaction with a web app.

If you're interested in learning more, peruse this nice [tutorial](https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251).